In [15]:
# ! pip install crewai crewai_tools openai langchain


In [16]:
import os
from dotenv import load_dotenv  # 导入python-dotenv

from crewai import Agent, Task, Crew, Process,LLM
from crewai_tools import (
    SerperDevTool,       # 网络搜索
    CSVSearchTool,      # 分析CSV数据

)

from openai import OpenAI
from pydantic import Field
from crewai.tools import tool


In [17]:
load_dotenv()
BASE_URL= "https://dashscope.aliyuncs.com/compatible-mode/v1"
ONLINE_MODEL_NAME ="qwen-plus"
api_key = (
            os.getenv("QWEN_API_KEY")
            or os.getenv("DASHSCOPE_API_KEY")
        ) 


In [18]:
product_name ="Samsung Z Flip7"

In [19]:

llm = LLM(
            base_url=BASE_URL,
            api_key=api_key,
            model="openai/"+ONLINE_MODEL_NAME,  # 本次使用的模型
            # temperature=0.7,  # 发散的程度
            # timeout=None,
            # max_retries=2,
        )




In [ ]:

search_tool = SerperDevTool()          # 网络搜索

In [21]:
# 创建市场研究员智能体
market_researcher = Agent(
    role="市场研究员",
    goal="收集并分析目标产品的市场数据和竞争格局",
    backstory="拥有5年市场研究经验，擅长分析产品定位和竞争优势，"
              "能够从复杂数据中提取有价值的市场洞察。",
    tools=[search_tool,],
    llm=llm,
    verbose=True
)



# 创建数据分析师智能体
data_analyst = Agent(
    role="数据分析师",
    goal="将市场数据可视化并提供数据支持",
    backstory="数据科学专家，擅长将市场数据转化为直观的图表和数据摘要，"
              "帮助团队理解数据背后的含义。",
    tools=[],
    llm=llm,
    verbose=True
)

# 创建内容创作者智能体
content_creator = Agent(
    role="内容创作者",
    goal="基于市场洞察创作吸引人的产品推广文案",
    backstory="资深文案策划，擅长将复杂的产品优势转化为简洁有力的推广语言，"
              "熟悉社交媒体和广告文案风格。",
  
    llm=llm,
    verbose=True
)

In [22]:
# 定义任务

task1 = Task(
    description=f"收集并分析'{product_name}'的市场数据，包括竞争对手、用户评价和市场趋势",
    expected_output="一份详细的市场分析报告，包含关键洞察和机会点",
    agent=market_researcher
)

task2 = Task(
    description=f"基于市场分析报告，为'{product_name}'创作3条适合社交媒体的推广文案，"
                "突出产品优势和差异化特点",
    expected_output="3条简洁有力、符合社交媒体风格的推广文案",
    agent=content_creator,
    context=[task1]  # 依赖任务1的结果
)

task3 = Task(
    description="总结市场分析的关键数据点，生成数据摘要",
    expected_output="一份包含核心数据和关键发现的数据摘要",
    agent=data_analyst,
    context=[task1]  # 依赖任务1的结果
)


In [23]:
# 创建智能体团队
crew = Crew(
    agents=[market_researcher, content_creator, data_analyst],
    tasks=[task1, task2, task3],
    process=Process.sequential  # 按顺序执行任务
)

# 启动团队协作
results = crew.kickoff()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 市场研究员                                                                                              │
│                                                                                                                 │
│  Task: 收集并分析'Samsung Z Flip7'的市场数据，包括竞争对手、用户评价和市场趋势                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 市场研究员                                                                                              │
│                                                                                                                 │
│  Thought: 首先，我需要收集有关 'Samsung Z Flip7'                                                                │
│  的市场数据，包括其竞争对手、用户评价和市场趋势。为此，我将从互联网上搜索相关信息开始。                         │
│  Thought: 我需要找到关于 'Samsung Z Flip7' 的市场数据，包括竞争对手、用户评价和市场趋势。                       │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"Samsung Z Flip7 \\u5e02\\u573a\\u6570\\u636e \\u7ade\\u4e89\\u5bf9\\u624b                │
│  \\u7528\\u6237\\u8bc4\\u4ef7 \\u5e02\\u573a\\u8d8b\\u52bf\"}"                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'Samsung Z Flip7 市场数据 竞争对手 用户评价 市场趋势', 'type': 'search', 'num':     │
│  10, 'engine': 'google'}, 'organic': [{'title': '三星将削减Galaxy Z Fold 7 和Flip 7 的产量，降幅近40% :         │
│  r/GalaxyFold', 'link':                                                                                         │
│  'https://www.reddit.com/r/GalaxyFold/comments/1hleyt1/yikes_samsung_reportedly_cutting_galaxy_z_fold_7/?tl=zh  │
│  -hans', 'snippet':                                                                                             │
│  '分析师早在2023年12月就预测了2024年整体销售的下降。这也包括其他具有更好摄像头、更大电池、“正常”长宽比和更薄机  │
│  身的折叠屏手机。他们 ...', 'position': 1}, {'title': '从第七代Galaxy                                           │
│  Z系列新品“挤爆牙膏”，看三星的市场进取之道', 'link':                                                            │
│  'https://ce.ofweek.com/2025-07/ART-2022112-8110-30666575.html', 'snippet': '此数据，不仅意味着三星Galaxy Z     │
│  Fold7相对于前代Z Fold6成功减重24克、减厚3.2毫米，同时也意味着三星成功从国产厂商手中夺走了全球最轻大折叠头衔。  │
│  图片.', 'position': 2}, {'title': 'Samsung Galaxy Z Flip7：搭載全新無邊際封面螢幕的口袋AI神機', 'link':        │
│  'https://news.samsung.com/tw/samsung-galaxy-z-flip7%EF%BC%9A%E6%90%AD%E8%BC%89%E5%85%A8%E6%96%B0%E7%84%A1%E9%  │
│  82%8A%E9%9A%9B%E5%B0%81%E9%9D%A2%E8%9E%A2%E5%B9%95%E7%9A%84%E5%8F%A3%E8%A2%8Bai%E7%A5%9E%E6%A9%9F',            │
│  'snippet': 'Galaxy Z Flip7的整體設計更為精實，機身小巧卻容納容量更大的電池與更寬廣的顯示螢幕。憑藉Galaxy Z     │
│  Flip系列歷來最強的電池續航，即使多看一部電影也不必 ...', 'position': 3}, {'title': 'Samsung Galaxy Z Flip7 |   │
│  The pocket-perfect Galaxy AI phone', 'link':                                                                   │
│  'https://www.samsung.com/my/smartphones/galaxy-z-flip7/?srsltid=AfmBOooeIqEdjtcCfJhDiT0NksSXM7xNMk_WRe69cOK1E  │
│  cMhjn-6qsdq', 'snippet': 'Missing: 市场 数据 竞争 对手 用户 评价 趋势', 'position': 4}, {'title': '三星Galaxy  │
│  Flip 7评测：内涵与众不同的小折叠 - 新浪财经', 'link':                                                          │
│  'http://t.cj.sina.cn/articles/view/5828676220/15b6a8a7c001019mea', 'snippet':                                  │
│  '在这两种折叠屏手机的形态设计上，他们几乎都要比竞争对手早了两年、甚至是更长的时间跨度。 ...                    │
│  首先，无论从外观设计、还是硬件“内涵”上来说，Galaxy Z Flip ...', 'position': 5}, {'title': 'Galaxy              │
│  Unpacked发布会亮点速览：Z Fold7与Flip7重新定义三星折叠 ...', 'link':                                           │
│  'https://www.eet-china.com/mp/a420819.html', 'snippet':                                                        │
│  '尽管2024年全球折叠屏手机出货量同比增长12%，达到1720万部，三星的市场份额却从54%下滑至45%，且预计2025年整体折   │
│  叠屏市场将不再增长。自第四代产品以来， ...', 'position': 6}, {'title':                                         │
│  '三星电子稳居全球智能手机市场首位Galaxy Z7系列预售点燃韩国本土 ...', 'link':                                   │
│  'https://www.ajudaily.com/view/20250715101247097', 'snippet':                                                  │
│  '今年4月，SKT因遭大规模黑客攻击，导致用户大幅流失，市场份额一度跌破40%。目前，SKT借助Galaxy                    │
│  Z7系列预售积极回笼用户，力求扭转颓势。与此同...                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 市场研究员                                                                                              │
│                                                                                                                 │
│  Thought: Thought: 我已经找到了一些关于 'Samsung Z Flip7'                                                       │
│  的市场数据，包括竞争对手、用户评价和市场趋势的信息。现在我需要进一步搜索以获取更多详细信息。                   │
│  Thought: 我需要更多的信息来分析 'Samsung Z Flip7' 的市场数据，包括竞争对手、用户评价和市场趋势。               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 市场研究员                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Samsung Galaxy Z Flip7 市场分析报告                                                                          │
│                                                                                                                 │
│  ## 1. 产品概述                                                                                                 │
│  三星 Galaxy Z Flip7                                                                                            │
│  是三星电子推出的第七代翻盖式折叠屏智能手机，延续了该系列小巧便携的设计理念，同时在硬件性能、电池续航和用户体   │
│  验上进行了多项升级。该产品于2025年发布，搭载了Exynos                                                           │
│  2500芯片，配备5000万像素主摄像头和更大的外屏，电池容量提升至4300mAh，续航能力显著增强。                        │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## 2. 市场趋势分析                                                                                             │
│                                                                                                                 │
│  ### 2.1 折叠屏手机市场整体增长放缓                                                                             │
│  根据2024年数据，全球折叠屏手机出货量同比增长12%，达到1720万部，但预计2025年市场将不再增长。三星的市场份额从54  │
│  %下滑至45%，表明其面临来自中国厂商的激烈竞争。                                                                 │
│                                                                                                                 │
│  ### 2.2 中国厂商的快速崛起                                                                                     │
│  中国厂商如华为、OPPO、vivo等，在折叠屏技术上取得了快速进展。他们推出的折叠手机在摄像头、电池容量、机身厚度等   │
│  方面已接近甚至超越三星产品，进一步压缩了三星的市场空间。                                                       │
│                                                                                                                 │
│  ### 2.3 消费者对轻薄与实用性的需求提升                                                                         │
│  Z                                                                                                              │
│  Flip系列凭借其小巧的折叠设计和便携性受到消费者欢迎。2025年市场数据显示，用户对轻薄机身和实用性（如悬停拍摄、A  │
│  I功能）的需求持续上升，这为Z Flip7提供了市场机会。                                                             │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## 3. 竞争对手分析                                                                                             │
│                                                                                                                 │
│  ### 3.1 主要竞争对手                                                                                           │
│  - **华为 P70                                                                                                   │
│  Pocket**：华为的折叠屏旗舰产品，采用国产折叠铰链技术，价格更具竞争力，同时搭载鸿蒙系统，生态体验更佳。         │
│  - **OPPO Find N3 Flip**：主打轻薄设计和大外屏，性价比高，吸引年轻消费者。                                      │
│  - **vivo X Flip2**：在铰链耐用性和屏幕折痕控制方面表现优异，用户体验良好。                 

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 内容创作者                                                                                              │
│                                                                                                                 │
│  Task: 基于市场分析报告，为'Samsung Z Flip7'创作3条适合社交媒体的推广文案，突出产品优势和差异化特点             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 内容创作者                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **#折叠美学 再进化**                                                                                        │
│  小巧机身，强大性能。Galaxy Z Flip7 以全球最轻薄设计，重新定义折叠屏手机！搭载 Exynos 2500                      │
│  芯片，5000万像素主摄+AI影像加持，拍照更聪明，生活更高效。便携与实力兼得，你的时尚智能新宠，非它莫属。          │
│                                                                                                                 │
│  2. **折叠，不只是形态，更是生活方式**                                                                          │
│  悬停拍摄、AI多任务处理，Galaxy Z Flip7 让科技为你服务。4300mAh                                                 │
│  大电池+智能省电，续航更持久。轻薄随行，智慧随身，Z世代的潮流选择，轻松拍出大片感！                             │
│                                                                                                                 │
│  3. **折叠屏，更耐用、更聪明的选择**                                                                            │
│  三星匠心打造，Z Flip7                                                                                          │
│  铰链更坚固，屏幕折痕更隐形。AI功能全面升级，从语音助手到智能拍摄，每一步都更懂你。轻薄有型，实力有料，折叠未   │
│  来，尽在掌握！                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 数据分析师                                                                                              │
│                                                                                                                 │
│  Task: 总结市场分析的关键数据点，生成数据摘要                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 数据分析师                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Samsung Galaxy Z Flip7 市场分析数据摘要                                                                      │
│                                                                                                                 │
│  ## 1. 产品核心数据                                                                                             │
│  - **发布年份**：2025年                                                                                         │
│  - **芯片**：Exynos 2500                                                                                        │
│  - **摄像头**：5000万像素主摄                                                                                   │
│  - **电池容量**：4300mAh                                                                                        │
│  - **设计特点**：翻盖式折叠屏、全球最轻薄设计                                                                   │
│  - **外屏尺寸**：更大外屏，提升实用性                                                                           │
│                                                                                                                 │
│  ## 2. 市场趋势数据                                                                                             │
│  - **全球折叠屏手机出货量（2024年）**：1720万部，同比增长12%                                                    │
│  - **预计2025年市场增长**：停滞                                                                                 │
│  - **三星市场份额变化**：从54%下滑至45%                                                                         │
│  - **主要市场趋势**：消费者对轻薄机身、悬停拍摄、AI功能需求上升                                                 │
│                                                                                                                 │
│  ## 3. 竞争对手与市场定位                                                                                       │
│                                                                                                                 │
│  ### 3.1 主要竞争对手                                                                                           │
│  | 品牌 | 产品 | 主要优势 |                                                                                     │
│  |------|------|----------|                                                                                     │
│  | 华为 | P70 Pocket | 鸿蒙系统生态、国产化供应链稳定 |                                                         │
│  | OPPO | Find N3 Flip | 大外屏、性价比高 |                                                                     │
│  | vivo | X Flip2 | 折痕控制技术领先 |                                                                          │
│                                                                                                                 │
│  ### 3.2 三星 Galaxy Z Flip7 竞争优势                                                                           │
│  - 全球最轻薄折叠设计                                                                                           │
│  - Exynos 2500芯片性能强                                                                                        │
│  - AI功能丰富，提升用户体验                                                                                     │
│                                                                                                                 │
│  ### 3.3 劣势                                                                                                   │
│  - 价格昂贵（通常在1200美元以上）                                             

In [24]:
results

CrewOutput(raw='# Samsung Galaxy Z Flip7 市场分析数据摘要  \n\n## 1. 产品核心数据  \n- **发布年份**：2025年  \n- **芯片**：Exynos 2500  \n- **摄像头**：5000万像素主摄  \n- **电池容量**：4300mAh  \n- **设计特点**：翻盖式折叠屏、全球最轻薄设计  \n- **外屏尺寸**：更大外屏，提升实用性  \n\n## 2. 市场趋势数据  \n- **全球折叠屏手机出货量（2024年）**：1720万部，同比增长12%  \n- **预计2025年市场增长**：停滞  \n- **三星市场份额变化**：从54%下滑至45%  \n- **主要市场趋势**：消费者对轻薄机身、悬停拍摄、AI功能需求上升  \n\n## 3. 竞争对手与市场定位  \n\n### 3.1 主要竞争对手  \n| 品牌 | 产品 | 主要优势 |\n|------|------|----------|\n| 华为 | P70 Pocket | 鸿蒙系统生态、国产化供应链稳定 |\n| OPPO | Find N3 Flip | 大外屏、性价比高 |\n| vivo | X Flip2 | 折痕控制技术领先 |\n\n### 3.2 三星 Galaxy Z Flip7 竞争优势  \n- 全球最轻薄折叠设计  \n- Exynos 2500芯片性能强  \n- AI功能丰富，提升用户体验  \n\n### 3.3 劣势  \n- 价格昂贵（通常在1200美元以上）  \n- 部分市场仍使用Exynos芯片（非高通）  \n\n## 4. 用户评价数据  \n### 4.1 正面反馈  \n- **便携性**：适合放入口袋或小型包中  \n- **摄像头表现**：夜间拍摄和AI优化表现突出  \n- **AI功能**：多任务处理和语音识别体验提升  \n- **续航能力**：4300mAh电池满足日常使用  \n\n### 4.2 负面反馈  \n- **价格偏高**：与竞争对手相比缺乏竞争力  \n- **铰链耐久性担忧**：部分用户对长期使用可靠性存疑  \n- **软件更新滞后**：Android 16和OneUI 8更新进度缓慢  \n\n## 5. 市场机会  \n-